In [29]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import datetime
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import scipy as sc
import plotly.graph_objects as go
import sqlite3

In [30]:
end_date = datetime.today()
start_date = end_date - timedelta(days=1500)
tickers = ["AAL","BABA","META","AAPL"]
weights = [20, 30, 30, 20]

In [31]:
# Import data
def getData(tickers:list, start_date:datetime, end_date:datetime):
    
    df = yf.download( tickers,
                    start=start_date,
                    end = end_date
                    )
    df = df['Adj Close']
    returns = df.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    return meanReturns, covMatrix

meanReturns, covMatrix = getData(tickers, start_date, end_date)

[*********************100%***********************]  4 of 4 completed


In [32]:
def portfolioPerformance(weights:list, meanReturns, covMatrix):
    returns = np.sum(meanReturns*weights)*252
    std = np.sqrt(
            np.dot(np.array(weights).T,np.dot(covMatrix, weights))
           )*np.sqrt(252)
    return returns, std



In [33]:
returns, std = portfolioPerformance(weights, meanReturns, covMatrix)
returns, std

(12.069408252486374, 33.85942110370829)

# Maximium Sharpe Ratio Portfolio

Highest reward vs risk

what each assets return and variance is going to be into the future and minimise risk while maximising our return. So we have to estimate these parameters, and one way of doing this is using historical stock price data.

In [34]:
def negativeSR(weights, meanReturns, covMatrix, riskFreeRate = 0):
    pReturns, pStd = portfolioPerformance(weights, meanReturns, covMatrix)
    return - (pReturns - riskFreeRate)/pStd

def maxSR(meanReturns, covMatrix, riskFreeRate = 0, constraintSet=(0,1)):
    "Minimize the negative SR, by altering the weights of the portfolio"
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix, riskFreeRate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = sc.optimize.minimize(negativeSR, numAssets*[1./numAssets], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

In [35]:
result = maxSR(meanReturns, covMatrix)
max_sr, max_weigths = result['fun'], result['x']
print(max_sr, max_weigths)

-1.1126353263709565 [2.49800181e-16 1.00000000e+00 1.46814065e-16 8.32667268e-16]


In [36]:
def portfolioVariance(weights, meanReturns, covMatrix):
    return portfolioPerformance(weights, meanReturns, covMatrix)[1]

def minimizeVariance(meanReturns, covMatrix, constraintSet=(0,1)):
    """Minimize the portfolio variance by altering the 
     weights/allocation of assets in the portfolio"""
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = sc.optimize.minimize(portfolioVariance, numAssets*[1./numAssets], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

In [37]:
min_var_result = minimizeVariance(meanReturns, covMatrix)
min_var, min_var_weights = min_var_result['fun'], min_var_result['x']
print(min_var, min_var_weights)

0.3160033604216835 [0.0826305  0.65548384 0.17564412 0.08624154]


## Creating the Efficient Frontier

In [38]:
def portfolioReturn(weights, meanReturns, covMatrix):
        return portfolioPerformance(weights, meanReturns, covMatrix)[0]


def efficientOpt(meanReturns, covMatrix, returnTarget, constraintSet=(0,1)):
    """For each returnTarget, we want to optimise the portfolio for min variance"""
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)

    constraints = ({'type':'eq', 'fun': lambda x: portfolioReturn(x, meanReturns, covMatrix) - returnTarget},
                    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    effOpt = sc.optimize.minimize(portfolioVariance, numAssets*[1./numAssets], args=args, method = 'SLSQP', bounds=bounds, constraints=constraints)
    return effOpt

In [39]:
def calculatedResults(meanReturns, covMatrix, riskFreeRate=0, constraintSet=(0,1)):
    """Read in mean, cov matrix, and other financial information
        Output, Max SR , Min Volatility, efficient frontier """
    # Max Sharpe Ratio Portfolio
    maxSR_Portfolio = maxSR(meanReturns, covMatrix)
    maxSR_returns, maxSR_std = portfolioPerformance(maxSR_Portfolio['x'], meanReturns, covMatrix)
    maxSR_returns, maxSR_std = round(maxSR_returns*100,2), round(maxSR_std*100,2)
    maxSR_allocation = pd.DataFrame(maxSR_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    maxSR_allocation.allocation = [round(i*100,0) for i in maxSR_allocation.allocation]
    
    # Min Volatility Portfolio
    minVol_Portfolio = minimizeVariance(meanReturns, covMatrix)
    minVol_returns, minVol_std = portfolioPerformance(minVol_Portfolio['x'], meanReturns, covMatrix)
    minVol_returns, minVol_std = round(minVol_returns*100,2), round(minVol_std*100,2)
    minVol_allocation = pd.DataFrame(minVol_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    minVol_allocation.allocation = [round(i*100,0) for i in minVol_allocation.allocation]

    # Efficient Frontier
    efficientList = []
    targetReturns = np.linspace(minVol_returns/100, maxSR_returns/100, 20)
    for target in targetReturns:
        efficientList.append(efficientOpt(meanReturns, covMatrix, target)['fun'])

    return maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList, targetReturns

In [40]:
print(calculatedResults(meanReturns, covMatrix))

(37.64, 33.83,       allocation
AAL          0.0
AAPL       100.0
BABA         0.0
META         0.0, 24.76, 31.6,       allocation
AAL          8.0
AAPL        66.0
BABA        18.0
META         9.0, [0.31600326288697766, 0.31606489560339696, 0.31625296643758644, 0.3165672500453528, 0.31700737085573455, 0.31757280589903913, 0.31826288723838037, 0.31907680701367847, 0.32001362760515795, 0.3210722712472966, 0.32225149847904266, 0.3235500346139437, 0.3249664431837602, 0.32649918637213504, 0.32814923375499166, 0.3299290083966475, 0.3318372144601137, 0.3338724611214703, 0.33603283817971685, 0.3383182546108385], array([0.2476    , 0.25437895, 0.26115789, 0.26793684, 0.27471579,
       0.28149474, 0.28827368, 0.29505263, 0.30183158, 0.30861053,
       0.31538947, 0.32216842, 0.32894737, 0.33572632, 0.34250526,
       0.34928421, 0.35606316, 0.36284211, 0.36962105, 0.3764    ]))


## Visualising the Efficient Frontier

In [41]:
def EF_graph(meanReturns, covMatrix, riskFreeRate=0, constraintSet=(0,1)):
    """Return a graph ploting the min vol, max sr and efficient frontier"""
    maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList, targetReturns = calculatedResults(meanReturns, covMatrix, riskFreeRate, constraintSet)

    #Max SR
    MaxSharpeRatio = go.Scatter(
        name='Maximium Sharpe Ratio',
        mode='markers',
        x=[maxSR_std],
        y=[maxSR_returns],
        marker=dict(color='red',size=14,line=dict(width=3, color='black'))
    )

    #Min Vol
    MinVol = go.Scatter(
        name='Mininium Volatility',
        mode='markers',
        x=[minVol_std],
        y=[minVol_returns],
        marker=dict(color='green',size=14,line=dict(width=3, color='black'))
    )

    #Efficient Frontier
    EF_curve = go.Scatter(
        name='Efficient Frontier',
        mode='lines',
        x=[round(ef_std*100, 2) for ef_std in efficientList],
        y=[round(target*100, 2) for target in targetReturns],
        line=dict(color='black', width=4, dash='dashdot')
    )

    data = [MaxSharpeRatio, MinVol, EF_curve]

    layout = go.Layout(
        title = 'Portfolio Optimisation with the Efficient Frontier',
        yaxis = dict(title='Annualised Return (%)'),
        xaxis = dict(title='Annualised Volatility (%)'),
        showlegend = True,
        legend = dict(
            x = 0.75, y = 0, traceorder='normal',
            bgcolor='#E2E2E2',
            bordercolor='black',
            borderwidth=2),
        width=800,
        height=600)
    
    fig = go.Figure(data=data, layout=layout)
    return fig.show()

In [43]:
EF_graph(meanReturns, covMatrix)